# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [48]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import h5py

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from new_functions import visualize2

# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "../../../Data/Raw/30SLight2") #images

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
print(ROOT_DIR)
print(IMAGE_DIR)


/Users/Tjoun/UCL/Dissertation/Motion_Prediction/src/MITMaskRCNN/Mask_RCNN
/Users/Tjoun/UCL/Dissertation/Motion_Prediction/src/MITMaskRCNN/Mask_RCNN/../../../Data/Raw/30SLight1


## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [3]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [ 0.1  0.1  0.2  0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_I

## Create Model and Load Trained Weights

In [4]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [43]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## Run Object Detection

In [9]:
# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]

image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

# Run detection
results = model.detect([image], verbose=0)

# Visualize results
r = results[0]
print(image.shape)
print(r.keys())
#print(results)

save_path = os.path.join(ROOT_DIR, "../../Data/")

f = h5py.File("{}mytestfile.hdf5".format(save_path), "w")
for k,v in r.items():
    print(k)
    f.create_dataset(k, data = v)
    
#visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                           # class_names, r['scores'])


#save_instances(image,save_path, r['rois'], r['masks'], r['class_ids'], 
        #                    class_names, r['scores'])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


(540, 930, 3)
dict_keys(['rois', 'class_ids', 'scores', 'masks'])


OSError: Unable to create file (unable to open file: name = '/Users/Tjoun/UCL/Dissertation/Motion_Prediction/src/MITMaskRCNN/Mask_RCNN/../../Data/mytestfile.hdf5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 602)

In [32]:
import colorsys
def make_colormap(N, bright = True):
        brightness = 1.0 if bright else 0.7
        hsv = [(i / N, 1, brightness) for i in range(N)]
        colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
        random.shuffle(colors)
        return colors

In [39]:
##### VISUALISE ONE WITH ID #########



save_path = os.path.join(ROOT_DIR, "../../../Data/")
col = make_colormap(500)


f = h5py.File("{}30SLight1_Tracked_Fewer.hdf5".format(save_path), "r")

for i in range(1,len(list(f.keys()))):
    frame = "frame{}".format(i)
    r = f[frame]
    image =r['image'].value


    #for k,v in r.items():
      #  print(k)
      #  f.create_dataset(k, data = v)

        
    #visualize2.save_instances(image, r['rois'].value, r['masks'].value, r['class_ids'].value, 
      #                         class_names, r['ID'].value,IDs= r['ID'].value, colormap = col )

    save_path2 = os.path.join(save_path, "FullSegmentation/30STracked/{}.jpg".format(frame))
    visualize2.save_instances(image,save_path2, r['rois'], r['masks'], r['class_ids'], 
                             class_names, r['ID'].value,IDs= r['ID'].value, colormap = col )

In [37]:
f.close()

In [ ]:
f = h5py.File("{}mytestfile.hdf5".format(save_path), "r")

In [ ]:
f.keys()

In [ ]:
dset = f['rois']

In [ ]:
dset.value
f.close()

In [49]:
print(IMAGE_DIR)

/Users/Tjoun/UCL/Dissertation/Motion_Prediction/src/MITMaskRCNN/Mask_RCNN/../../../Data/Raw/30SLight2


In [50]:
# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]

In [51]:
print(file_names)

['.DS_Store', '30SLight1001.jpg', '30SLight1002.jpg', '30SLight1003.jpg', '30SLight1004.jpg', '30SLight1005.jpg', '30SLight1006.jpg', '30SLight1007.jpg', '30SLight1008.jpg', '30SLight1009.jpg', '30SLight1010.jpg', '30SLight1011.jpg', '30SLight1012.jpg', '30SLight1013.jpg', '30SLight1014.jpg', '30SLight1015.jpg', '30SLight1016.jpg', '30SLight1017.jpg', '30SLight1018.jpg', '30SLight1019.jpg', '30SLight1020.jpg', '30SLight1021.jpg', '30SLight1022.jpg', '30SLight1023.jpg', '30SLight1024.jpg', '30SLight1025.jpg', '30SLight1026.jpg', '30SLight1027.jpg', '30SLight1028.jpg', '30SLight1029.jpg', '30SLight1030.jpg', '30SLight1031.jpg', '30SLight1032.jpg', '30SLight1033.jpg', '30SLight1034.jpg', '30SLight1035.jpg', '30SLight1036.jpg', '30SLight1037.jpg', '30SLight1038.jpg', '30SLight1039.jpg', '30SLight1040.jpg', '30SLight1041.jpg', '30SLight1042.jpg', '30SLight1043.jpg', '30SLight1044.jpg', '30SLight1045.jpg', '30SLight1046.jpg', '30SLight1047.jpg', '30SLight1048.jpg', '30SLight1049.jpg', '30SLi

In [ ]:
for file_name in file_names :
    print(file_name)

In [41]:
N_FRAMES = len(file_names)
IM_DIM = skimage.io.imread(os.path.join(IMAGE_DIR, file_names[0])).shape
print(IM_DIM)

(540, 930, 3)


In [55]:
i=1


#/
##Class IDs
##Frame
###Image
###r
#class_names_arr = np.array(class_names)
#print(type(class_names_arr))
#print(class_names_arr)
save_path = os.path.join(ROOT_DIR, "../../../Data/")
f = h5py.File("{}30SLight2.hdf5".format(save_path), "w")
f.create_dataset("class_names", data = [np.string_(j) for j in class_names])


for file_name in file_names:
    if(file_name ==".DS_Store"):
        continue
    image = skimage.io.imread(os.path.join(IMAGE_DIR, file_name))
    f.create_dataset("frame{}/image".format(i), data = image)
    # Run detection
    results = model.detect([image], verbose=0)

    # Visualize results
    r = results[0]

    for k,v in r.items():
       
        f.create_dataset("frame{}/{}".format(i,k), data = v)
    
    save_path = os.path.join(ROOT_DIR, "../../../Data/FullSegmentation/30SLight2/{}".format(file_name))
    visualize2.save_instances(image,save_path, r['rois'], r['masks'], r['class_ids'], 
                        class_names, r['scores'])
    
    i+=1
    if(i%20 == 0 ):
        print(i)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


20
40
60
80
100
120
140
160
180
200
220
240
260
280


In [52]:
f.close()

In [ ]:
f.keys()

In [ ]:

print(np.shape(r['rois']))
print(np.shape(r['class_ids']))
print(np.shape(r['scores']))
print(np.shape(r['masks']))
print(r)